In [ ]:
import cv2
import time

# ============================================================
# STEP 1: LOAD PRE-TRAINED HAAR CASCADE MODELS
# ============================================================
# Haar cascades are classical ML models trained on
# positive (face/eye) and negative (non-face) images

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

eye_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_eye.xml"
)

# Ensure cascade files are loaded correctly
if face_cascade.empty() or eye_cascade.empty():
    raise IOError("Failed to load Haar Cascade XML files")

# ============================================================
# STEP 2: INITIALIZE VIDEO CAPTURE (WEBCAM)
# ============================================================
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot access webcam")

# Variables for FPS calculation
prev_time = 0

print("Press 'q' to exit")

# ============================================================
# STEP 3: PROCESS VIDEO FRAME BY FRAME
# ============================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --------------------------------------------------------
    # Convert frame to grayscale
    # Haar cascades work on intensity values only
    # --------------------------------------------------------
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Improve contrast for better detection
    gray = cv2.equalizeHist(gray)

    # --------------------------------------------------------
    # STEP 4: FACE DETECTION
    # --------------------------------------------------------
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,        # Controls image scaling
        minNeighbors=6,         # Higher = fewer false positives
        minSize=(60, 60),       # Minimum face size
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # --------------------------------------------------------
    # STEP 5: PROCESS EACH DETECTED FACE
    # --------------------------------------------------------
    for (x, y, w, h) in faces:

        # Draw bounding box around face
        cv2.rectangle(
            frame,
            (x, y),
            (x + w, y + h),
            (255, 0, 0),
            2
        )

        # Region of Interest (ROI) for eyes
        face_gray = gray[y:y + h, x:x + w]
        face_color = frame[y:y + h, x:x + w]

        # ----------------------------------------------------
        # STEP 6: EYE DETECTION (ONLY INSIDE FACE ROI)
        # ----------------------------------------------------
        eyes = eye_cascade.detectMultiScale(
            face_gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(20, 20)
        )

        for (ex, ey, ew, eh) in eyes:
            center = (ex + ew // 2, ey + eh // 2)
            radius = int((ew + eh) * 0.25)

            cv2.circle(
                face_color,
                center,
                radius,
                (0, 255, 0),
                2
            )

    # --------------------------------------------------------
    # STEP 7: FPS CALCULATION
    # --------------------------------------------------------
    current_time = time.time()
    fps = int(1 / (current_time - prev_time)) if prev_time else 0
    prev_time = current_time

    cv2.putText(
        frame,
        f"FPS: {fps}",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 255),
        2
    )

    # --------------------------------------------------------
    # STEP 8: DISPLAY OUTPUT
    # --------------------------------------------------------
    cv2.imshow("Classical Face & Eye Detection (Haar Cascades)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ============================================================
# STEP 9: RELEASE RESOURCES
# ============================================================
cap.release()
cv2.destroyAllWindows()


Press 'q' to exit
